In [5]:
import tensorflow as tf
import numpy as np

NUM_STATES = 10
NUM_ACTIONS = 2
Gamma = 0.8

Define one-hot function that returns a state's index to a one-hot vector

In [18]:
def one_hot_state(index):
    array = np.zeros(NUM_STATES)
    array[index] = 1.
    return array

Define state's reward, if index = 4, reward = 1, else =0

In [19]:
state_reward = np.zeros(NUM_STATES)
state_reward[4] = 1.
print("state's reward vect:", state_reward)

state's reward vect: [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]


In Deep Q learning we try to minimize the difference between Qtarget and Qprediction. Now we create 2 layer NN, with input's size =  NUM_STATES and output's size = NUM_ACTIONS

In [43]:
session = tf.Session()
state = tf.placeholder("float", [None, NUM_STATES])
Qtarget = tf.placeholder("float", [None, NUM_ACTIONS])
#Qtarget is the targeted Q-value of actions

hidden_weights = tf.Variable(tf.constant(0., shape = [NUM_STATES,NUM_ACTIONS]))

Qprediction = tf.matmul(state,hidden_weights)
#Qprediction is the predicted Q-value of actions

loss  = tf.reduce_mean(tf.square(Qprediction - Qtarget  ))
train_operation = tf.train.AdamOptimizer(0.1).minimize(loss)

session.run(tf.global_variables_initializer())

Training Procedure

In [44]:
for i in range(50):
    state_batch = []
    reward_batch = []
    
    # create a batch of states (input) and a batch of reward (Qtarget/output) for trainning
    for state_index in range(NUM_STATES):
        state_batch.append(one_hot_state(state_index))
        
        #all possible new state s'
        go_left_index = (state_index - 1) % NUM_STATES
        go_right_index = (state_index + 1) % NUM_STATES
        
        #compute the pred value of Q(s',a') for all posible s' and a'
        go_left_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_left_index)]})[0]
        go_right_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_right_index)]})[0]
        
        #Qtarget(s') = rew + gamma* max Q(s',a') /over a'
        target_rew = [state_reward[go_left_index] + Gamma*np.max(go_left_Qval),
                     state_reward[go_right_index] +  Gamma*np.max(go_right_index)]
        
        reward_batch.append(target_rew)
    
    #start train linear model
    session.run(train_operation,
               feed_dict= {state: state_batch, Qtarget: reward_batch})
    
    #print out the learned result
    print([state_reward[x] + np.max(session.run(Qprediction, feed_dict={state: [one_hot_state(x)]}))
           for x in range(NUM_STATES)])

[0.099999368190765381, 0.099999688565731049, 0.099999800324440002, 0.099999897181987762, 1.0999998673796654, 0.099999897181987762, 0.099999904632568359, 0.099999919533729553, 0.09999992698431015, 0.0]
[0.19881114363670349, 0.1995868980884552, 0.19976025819778442, 0.19989678263664246, 1.1998722553253174, 0.19989678263664246, 0.19991345703601837, 0.19992552697658539, 0.19993466138839722, 0.074408724904060364]
[0.29512625932693481, 0.29841196537017822, 0.29909563064575195, 0.2996172308921814, 1.29952472448349, 0.2996172308921814, 0.29967981576919556, 0.29972493648529053, 0.29975897073745728, 0.15470406413078308]
[0.3870815634727478, 0.39605873823165894, 0.39779916405677795, 0.39908486604690552, 1.3988598883152008, 0.39908486604690552, 0.39923638105392456, 0.39934521913528442, 0.39942705631256104, 0.21525681018829346]
[0.47218221426010132, 0.49203377962112427, 0.4956396222114563, 0.49821865558624268, 1.4977733790874481, 0.49821865558624268, 0.49851709604263306, 0.49873062968254089, 0.49889

In [7]:
import tensorflow as tf
import numpy as np

NUM_STATES = 10
NUM_ACTIONS = 2
Gamma = 0.2

def one_hot_state(index):
    array = np.zeros(NUM_STATES)
    array[index] = 1.
    return array

state_reward = [(x == 4) for x in range(NUM_STATES)]

session = tf.Session()
state = tf.placeholder("float", [None, NUM_STATES])
Qtarget = tf.placeholder("float", [None, NUM_ACTIONS])
#Qtarget is the targeted Q-value of actions

hidden_weights = tf.Variable(tf.constant(0., shape=[NUM_STATES, NUM_ACTIONS]))

Qprediction = tf.matmul(state,hidden_weights)
#Qprediction is the predicted Q-value of actions

loss  = tf.reduce_mean(tf.square(Qprediction - Qtarget  ))
train_operation = tf.train.AdamOptimizer(0.1).minimize(loss)

session.run(tf.global_variables_initializer())

for i in range(500):
    state_batch = []
    reward_batch = []
    
    # create a batch of states (input) and a batch of reward (Qtarget/output) for trainning
    for state_index in range(NUM_STATES):
        state_batch.append(one_hot_state(state_index))
        
        #all possible new state s'
        go_left_index = (state_index - 1) % NUM_STATES
        go_right_index = (state_index + 1) % NUM_STATES
        
        #compute the pred value of Q(s',a') for all posible s' and a'
        go_left_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_left_index)]})[0]
        go_right_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_right_index)]})[0]
        #go_left_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_left_index)]})
      
        #go_right_Qval = session.run(Qprediction, feed_dict={state: [one_hot_state(go_right_index)]})
        
        #Qtarget(s,a) = rew + gamma* max Q(s',a') /over a'
        target_rew = [state_reward[go_left_index] + Gamma*np.max(go_left_Qval),
                     state_reward[go_right_index] +  Gamma*np.max(go_right_Qval)]
        
        reward_batch.append(target_rew)
    
    #start train linear model
    session.run(train_operation,
               feed_dict= {state: state_batch, Qtarget: reward_batch})
    
    #print out the learned result
    if i % 10 == 1:
        print([state_reward[x] + Gamma*np.max(session.run(Qprediction, feed_dict={state: [one_hot_state(x)]}))
           for x in range(NUM_STATES)])

[0.0, 0.0, 0.014880333840847016, 0.03991737961769104, 1.014880333840847, 0.03991737961769104, 0.014880333840847016, 0.0, 0.0, 0.0]
[0.0074131056666374212, 0.0077257268130779266, 0.041042417287826538, 0.2138674020767212, 1.0410424172878265, 0.2138674020767212, 0.041042417287826538, 0.0077257268130779266, 0.0074131056666374212, 0.0074364297091960912]
[0.0077425584197044379, 0.0081842511892318733, 0.054672271013259888, 0.2606968879699707, 1.0546722710132599, 0.2606968879699707, 0.054672271013259888, 0.0081842511892318733, 0.0077425584197044379, 0.0031552545726299288]
[0.0059305492788553245, 0.0069371566176414491, 0.040653026103973394, 0.20981717109680176, 1.0406530261039735, 0.20981717109680176, 0.040653026103973394, 0.0069371566176414491, 0.0059305492788553245, 0.0013544321991503239]
[0.0042029205709695821, 0.0057020824402570728, 0.038462829589843754, 0.18911731243133545, 1.0384628295898437, 0.18911731243133545, 0.038462829589843754, 0.0057020824402570728, 0.0042029205709695821, 0.000847

[0.0016666665673255921, 0.0083333328366279602, 0.041666665673255922, 0.20833332538604737, 1.0416666656732558, 0.20833332538604737, 0.041666665673255922, 0.0083333328366279602, 0.0016666665673255921, 0.00033333336468786004]
[0.0016666660085320473, 0.0083333335816860206, 0.041666665673255922, 0.20833332538604737, 1.0416666656732558, 0.20833332538604737, 0.041666665673255922, 0.0083333335816860206, 0.0016666660085320473, 0.00033333294559270146]
[0.001666666753590107, 0.0083333328366279602, 0.041666668653488163, 0.20833332538604737, 1.0416666686534881, 0.20833332538604737, 0.041666668653488163, 0.0083333328366279602, 0.001666666753590107, 0.00033333345782011748]
[0.0016666682437062265, 0.0083333328366279602, 0.041666665673255922, 0.20833332538604737, 1.0416666656732558, 0.20833332538604737, 0.041666665673255922, 0.0083333328366279602, 0.0016666682437062265, 0.00033333455212414265]
[0.001666667126119137, 0.0083333320915699016, 0.041666662693023687, 0.20833332538604737, 1.0416666626930238, 0